# Bilsulfite Pipeline

![bisulfite_pipeline](https://ars.els-cdn.com/content/image/1-s2.0-S0168165617315936-gr1_lrg.jpg)

## 1.1 QC Step

* 2.1 Install following softwares

FastQC : FastQC aims to provide a simple way to do some quality control checks on raw sequence data coming from high throughput sequencing pipelines. It provides a modular set of analyses which you can use to give a quick impression of whether your data has any problems of which you should be aware before doing any further analysis.

MultiQC : A modular tool to aggregate results from bioinformatics analyses across many samples into a single report.

Pre Alignment QC Report can be viewed at:
https://htmlpreview.github.io/?https://github.com/cjgunase/CoRSIV/blob/master/files/multiqc_report.html

## 1.2 Split reads in each FASTQ file in to 1M read files for parallel processing


each sample is about 40GB file, so it will be very difficult to process unless with large amount of memory in each compute node.so I split each file in to small 10M read files and processed parellel for the efficiency.

```bash
#!/bin/sh

#PBS -N fastqc
#PBS -l nodes=2:ppn=4
#PBS -l walltime=6:00:00
#PBS -q dque
#PBS -e . -o .

# This job's temporary working directory. You may also work in any of your
# home directories
echo Working directory is $PBS_O_WORKDIR
echo Running on host `hostname`
echo Time is `date`
echo Directory is `pwd`
echo This job runs on the following processors:
echo "PBS_NODEFILE=" $PBS_NODEFILE

cd /store1_e/waterland/gunasekara/GTeX_File_System/raw_fq/GTEX-13OW7-0826-SM-D5A5L/

#Args: Input Fastq files

IN_FILE=$1
echo $IN_FILE

mkdir -p split

e=$(echo $IN_FILE|cut -f1,2 -d".")
echo $e

#zcat $IN_FILE | split --verbose -l 40000000 -d -a 4 --filter="pigz -p 2 -c > split/\$FILE.gz" - split.10m.$e
zcat $IN_FILE | split --verbose -l 40000000 -d -a 4 --filter="pigz -p 2 -c > $FILE.gz" - split.10m.$e
```



## 1.3 TrimGalore 
```shell
#!/bin/sh

#PBS -N bismark
#PBS -l nodes=1:ppn=8
#PBS -l walltime=4:00:00
#PBS -q dque
#PBS -o /store1_e/waterland/gunasekara/
#PBS -e /store1_e/waterland/gunasekara/

DIR_NAME=$(dirname $1)

mkdir $DIR_NAME/trimmed

OUTPUT_DIR=$DIR_NAME/trimmed

#trim_galore --paired -q 20 --length 50 -o $OUTPUT_DIR $1 $2
trim_galore --paired -o $OUTPUT_DIR $1 $2
```
each file is submitted process in a loop using correct pairing

```python
import glob
import os

read1 = sorted(glob.glob("./split.10m.GTEX-S7SE-2526-SM-D5A5E_1*"))
read2 = sorted(glob.glob("./split.10m.GTEX-S7SE-2526-SM-D5A5E_2*"))

for fqFile in list(range(31,70)):
    cmd = "sbatch /home/gunasekara/scripts/TrimG.sh " + read1[fqFile] + " " + read2[fqFile]
    print(cmd)
    os.system(cmd)

```

## 2 Align to hg38 [Reference Genome](https://en.wikipedia.org/wiki/Reference_genome)

### 2.1 Download and index the hg38 reference genome


### 2.2 align reads

Description of the parameters used:

-q : The query input files (specified as

--bowtie2 : Default: ON. Uses Bowtie 2 instead of Bowtie 1. Bismark limits Bowtie 2 to only perform end-to-end alignments, i.e. searches for alignments involving all read characters (also called untrimmed or unclipped alignments). Bismark assumes that raw sequence data is adapter and/or quality trimmed where appropriate. Both small (.bt2) and large (.bt2l) Bowtie 2 indexes are supported.

-p : This options if to paralleliztion of bowtie2

[genome folder] in this example, the pathe to the genome folder used by Jack is given.

-B : The base name of the .BAM file generated from the Bismark.

--multicore : this option works when -B is not specified. Use this to set the programm to run in parallel.
-1: read 1 fastq file
-2: read 2 fastq file

** Note that, if you are planning to use bisSNP later, add the --rg_tag to bismark. The the read groups will be tagged with some information about the sequecing for later use.


```shell

#!/bin/bash

#SBATCH --time=2-00:00 -n24 -p bynode

# Arg order: First_read.fq Second_read.fq Read_Group_string

starttime=$(date +"%s")
echo $(date -u -d @${starttime})


DIR_NAME=$(dirname $1)

mkdir "$DIR_NAME"/aligned

OUT_DIR=$DIR_NAME/aligned

echo "Output directory: $OUT_DIR"

bismark --multicore 6 -q -o $OUT_DIR --genome /home/scott/genomes/human/gencode -1 $1 -2 $2

endtime=$(date +"%s")
echo $(date -u -d @${endtime})
echo "Time elapsed" $(date -u -d @$(($endtime-$starttime)) +"%T")

```

### 2.3 split each bam file in to chromosomes

```python
####################################
# Because this is executing 24 jobs simultaneously it is
# best to execute this as a batch script on the bynode queue
# as it should use all of the available cores.
#
# Usage: "python3 split_by_chromosome.py [-h] [--chr] input.bam"
#   Input bam must be coordinate sorted and have a .bai index file
# But consider starting this with an sbatch job script
#
# Samtools must be installed and present on the PATH
#
####################################

import subprocess
import time
import sys
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("input_bam", help="Input bam file to split, file must have index present")
parser.add_argument("--chr", help="Set if input bam has chromosome IDs beginning with 'chr'", action="store_true")

args = parser.parse_args()

if args.chr:
    chromosomes = ["chr1", "chr2", "chr3", "chr4", "chr5", "chr6", "chr7", "chr8", "chr9", "chr10",
               "chr11", "chr12", "chr13", "chr14", "chr15", "chr16", "chr17", "chr18", "chr19",
               "chr20", "chr21", "chr22", "chrX", "chrY"]
else:
    chromosomes = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10",
                   "11", "12", "13", "14", "15", "16", "17", "18", "19",
                   "20", "21", "22", "X", "Y"]


# Get the input file from the command line
input_bam = args.input_bam
file_name = input_bam.split('.')[0]
print("Input file is %s" % input_bam)
sys.stdout.flush()  # This is used to force a write to stdout on slurm

# create list to hold all spawned job references
procs = []

# Start splitting every chromosome simultaneously
for chromosome in chromosomes:  # Make sure this is set for testing or production
    if args.chr:
        output_bam = file_name + '.' + chromosome + ".bam"
    else:
        output_bam = file_name + '.chr' + chromosome + ".bam"

    command = "samtools view -b %s %s > %s" % (input_bam, chromosome, output_bam)
    print("Command is: " + command)
    print("Output file is %s" % output_bam)
    p = subprocess.Popen("samtools view -b %s %s > %s" % (input_bam, chromosome, output_bam), shell=True)
    procs.append(p)
    sys.stdout.flush()

# Loop over all of the spawned jobs checking for completion
while True:
    counter = 0
    for p in procs:
        p.poll()
        print (p.returncode)
        sys.stdout.flush()
        if p.returncode is not None:
            counter += 1

    # If all are completed break the loop to end the python script
    if counter == len(chromosomes):  # Make sure this is set for testing or production
        break

    # Wait a little bit and check again
    else:
        print("Waiting 1 minute and check job status again")
        sys.stdout.flush()
        time.sleep(60)
        
        ```
```bash        
#!/bin/bash

#SBATCH --time=16:00:00 -n24 -p bynode

#Args: Input_file.bam [--chr]
# Use optional --chr if chromosome IDs start with 'chr'
python3 split_bam_by_chromosome.py $1

```
## 2.4 merge bam files to each chrosome and create 24 files chr1-22,X,Y

```bash
 #!/bin/sh

#PBS -N merge_bam_by_chr
#PBS -l nodes=1:ppn=4
#PBS -l walltime=4:00:00
#PBS -q dque

cd $2

# Get file name
FILE=$(basename $1)

# get the working directiory of the file
INPUT_DIR=$(dirname $1)

# create a sorted direction inside if it doesn't exist
mkdir -p "$INPUT_DIR"/gathered

# declare path for output file
OUTPUT_FILE="$INPUT_DIR"/gathered/"${PWD##*/}"."$FILE".gathered.bam

METRICS="$INPUT_DIR"/gathered/"$FILE".metrics.txt

echo $FILE
echo $OUT_FILE
echo $METRICS

bamtools merge -list $1 -out $OUTPUT_FILE
#$1 is the file with contains list of files to merged to single chromosome

```
### At the end of this pipeline, we have 24 files for each sample, 720 files total.

# 3 Remove PCR bias- Deduplication

# 4 Bismark methylation extraction

# 5 Bismark important reports